# 1. Data Layer – Preprocessing & Feature Engineering

This layer serves as the foundation of the project by preparing and transforming raw TCS stock data into a clean, enriched format suitable for exploratory data analysis and machine learning.

---

### Datasets Loaded
- `TCS_stock_history.csv` – Historical OHLCV data (Open, High, Low, Close, Volume)
- `TCS_stock_action.csv` – Corporate actions (e.g., splits, dividends)
- `TCS_stock_info.csv` – Stock-level metadata and reference information

---

### Data Cleaning & Date Parsing
- Converted `Date` columns to datetime objects.
- Removed rows with missing or null values across all datasets.
- Extracted date-related features: **Year, Month, Quarter, Day, Weekday**.

---

### Feature Engineering
- **Daily Return:** Day-to-day percentage change in Close price.
- **Cumulative Return:** Compounded product of daily returns to show investment growth.
- **MACD & Signal Line:** Technical indicators used to analyze momentum shifts.
- **RSI (Relative Strength Index):** Helps identify overbought/oversold market conditions.
- **52W High:** Highest stock price in the last 252 trading days (approx. 1 year).

---

### Lag Features & Rolling Statistics (for ML models)
- `Lag_1`, `Lag_2`: Previous Close prices (1-day and 2-day lag).
- `Rolling_Mean_7`: 7-day rolling average of Close price.
- `Rolling_Std_7`: 7-day rolling standard deviation of Close price.

These features are essential for time series regression and forecasting models like Random Forest, XGBoost, and LSTM.

---

### Outputs Saved
- `cleaned_TCS_stock_history.csv`
- `cleaned_TCS_stock_action.csv`
- `cleaned_TCS_stock_info.csv`

These files are now enriched, cleaned, and ready for downstream EDA and modeling.

---


In [1]:
import pandas as pd
import numpy as np

# 📥 Load datasets
history_df = pd.read_csv("data/TCS_stock_history.csv")
action_df = pd.read_csv("data/TCS_stock_action.csv")
info_df = pd.read_csv("data/TCS_stock_info.csv")

# 📅 Convert Date columns
history_df['Date'] = pd.to_datetime(history_df['Date'], errors='coerce')
action_df['Date'] = pd.to_datetime(action_df['Date'], errors='coerce')

# 📆 DateTime features
history_df['Year'] = history_df['Date'].dt.year
history_df['Month'] = history_df['Date'].dt.month
history_df['Quarter'] = history_df['Date'].dt.quarter
history_df['Day'] = history_df['Date'].dt.day
history_df['Weekday'] = history_df['Date'].dt.day_name()

# ─────────────────────────────────────────────
# 📈 Feature Engineering

# Daily Return & Cumulative Return
history_df['Daily Return'] = history_df['Close'].pct_change()
history_df['Cumulative Return'] = (1 + history_df['Daily Return']).cumprod()

# MACD & Signal Line
ema_short = history_df['Close'].ewm(span=12, adjust=False).mean()
ema_long = history_df['Close'].ewm(span=26, adjust=False).mean()
history_df['MACD'] = ema_short - ema_long
history_df['Signal_Line'] = history_df['MACD'].ewm(span=9, adjust=False).mean()

# RSI
delta = history_df['Close'].diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)
avg_gain = pd.Series(gain).rolling(window=14).mean()
avg_loss = pd.Series(loss).rolling(window=14).mean()
rs = avg_gain / avg_loss
history_df['RSI'] = 100 - (100 / (1 + rs))

# 52-Week High (approx. 252 trading days)
history_df['52W_High'] = history_df['High'].rolling(window=252).max()


# ─────────────────────────────────────────────
# 🔁 Lag Features & Rolling Statistics (for XGBoost)
history_df['Lag_1'] = history_df['Close'].shift(1)
history_df['Lag_2'] = history_df['Close'].shift(2)
history_df['Rolling_Mean_7'] = history_df['Close'].rolling(7).mean()
history_df['Rolling_Std_7'] = history_df['Close'].rolling(7).std()

# 🧹 Final Cleaning
history_df.dropna(inplace=True)
action_df.dropna(inplace=True)
info_df.dropna(how='all', inplace=True)

# 🧪 Prepare data for ML (optional here but useful in model layer)
features = [
    'Open', 'High', 'Low', 'Volume',
    'MACD', 'Signal_Line', 'RSI',
    'Lag_1', 'Lag_2', 'Rolling_Mean_7', 'Rolling_Std_7'
]
X = history_df[features]
y = history_df['Close']

# 💾 Save cleaned & engineered data
history_df.to_csv("data/cleaned_TCS_stock_history.csv", index=False)
action_df.to_csv("data/cleaned_TCS_stock_action.csv", index=False)
info_df.to_csv("data/cleaned_TCS_stock_info.csv", index=False)

# ✅ Preview
history_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,...,Daily Return,Cumulative Return,MACD,Signal_Line,RSI,52W_High,Lag_1,Lag_2,Rolling_Mean_7,Rolling_Std_7
251,2004-09-01,92.014959,92.479681,91.420113,91.819778,19935544,0.0,0.0,2004,9,...,-0.000202,3.110517,5.292094,4.222769,74.407190,111.412424,91.838310,91.713791,90.471661,2.297819
252,2004-09-02,92.005667,93.371945,91.643181,92.355133,21356352,0.0,0.0,2004,9,...,0.005830,3.128653,5.369806,4.452176,74.512148,111.412424,91.819778,91.838310,91.431113,0.951189
253,2004-09-03,93.502044,102.238816,92.048393,92.745476,9869856,0.0,0.0,2004,9,...,0.004227,3.141876,5.400635,4.641868,85.555783,111.412424,92.355133,91.819778,91.562695,1.070844
254,2004-09-06,96.652855,96.652855,92.284469,92.466637,9038672,0.0,0.0,2004,9,...,-0.003006,3.132430,5.341000,4.781694,84.540117,111.412424,92.745476,92.355133,91.773279,1.084885
255,2004-09-07,96.197402,96.197402,92.479628,92.535393,5772232,0.0,0.0,2004,9,...,0.000744,3.134759,5.238897,4.873135,84.572909,111.412424,92.466637,92.745476,92.210645,0.411539


---